In [2]:
from sklearn.datasets import fetch_20newsgroups
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import defaultdict
import gensim
from InitKmeans import InitKmeans
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
     
def pre_processData(newsgroups_train):
    tokenizer = RegexpTokenizer(r'\w+')
    for i in range(len(newsgroups_train)):
        newsgroups_train[i] = newsgroups_train[i].lower()
        #newsgroups_train[i] = tokenizer.tokenize(newsgroups_train[i])
    #newsgroups_train = [[token for token in doc if not token.isdigit()] for doc in newsgroups_train]
    newsgroups_train = [doc.split(' ') for doc in newsgroups_train]
    return newsgroups_train
from sklearn.datasets import fetch_20newsgroups
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import defaultdict
import gensim
from InitKmeans import InitKmeans
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
import sys  
import json
if __name__ == "__main__":
    newsgroups_train = list()
    news_labels = {}
    docID_username = {}
    
    i = 0
    file_path="D:/part_news2_biterm.txt"
    with open(file_path) as fp:
        lines = fp.read().split("\n")
        for line in lines:
            if line:
                text = json.loads(line)["bitermText"].strip()
                label = json.loads(line)["clusterNo"]
                newsgroups_train.append(text)
                news_labels[i] = label
                i+=1
    fp.close()
    
    corpus = pre_processData(newsgroups_train)

D:\anacoda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
doc_biterm = {}
for docID in range(len(corpus)):
    words = corpus[docID]
    duplicate_biterm = []
    doc_biterm[docID] = {}
    for biterm in words:
        hash_biterm = set(biterm.split(','))
        if hash_biterm not in duplicate_biterm:
            duplicate_biterm.append(hash_biterm)
            doc_biterm[docID][biterm] = 1
        else:
            if biterm in doc_biterm[docID].keys():
                doc_biterm[docID][biterm] += 1
            else:
                doc_biterm[docID][biterm] = 1
doc_words = {}
i_biterm = 0
for docID in range(len(corpus)):
    doc_words[docID] = {}
    words = corpus[docID]
    for biterm in words:
        for word in biterm.split(','):
            if word not in doc_words[docID].keys():
                doc_words[docID][word] = 0
            doc_words[docID][word] += 1

In [19]:
alpha_NMI = {}
alpha_topics = {}
for alpha in np.arange(0.5,5.5,0.5):
    alpha = np.around(alpha,decimals=1)
    docID_assign_z = {}
    m_z = {}
    n_z = {}
    n_w = {}
    n_b = {}
    Topics = []
    V = set()
    D = set()
    #alpha = 1
    beta = 0.02
    total_iter = 10
    for iter in range(total_iter):
        for docID in range(len(corpus)):
            if docID % 1000 == 0:
                print("iter: ",iter," ,docID ",docID)
                print("len(Topics) ",len(Topics))
            words = corpus[docID]
            D.discard(docID)
            if docID in docID_assign_z.keys():
                before_k = docID_assign_z[docID]
                m_z[before_k].discard(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        n_z[before_k][word] -= 1
                        n_w[before_k] -=1
            else:
                before_k = -1
            if len(D) == 0 and len(V) == 0:
                choose_k = 0
                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)
            else:
                log_pro = []
                for k in Topics:
                    pro_k = len(m_z[k])
                    if pro_k != 0:
                        i = 0
                        for biterm in words:
                            each_word = biterm.strip().split(',')
                            for word in each_word:
                                if word not in n_z[k].keys():
                                    n_z[k][word] = 0
                            for j in range(doc_biterm[docID][biterm]):
                                pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                i += 1

                    if pro_k == 0:
                        pro_k = sys.float_info.min
                    log_pro.append(pro_k)

                pro_new_k = alpha*(len(D))
                i = 0
                for biterm in words:
                    for j in range(doc_biterm[docID][biterm]):
                        pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                        i += 1

                if pro_new_k == 0:
                    pro_new_k = sys.float_info.min        
                log_pro.append(pro_new_k)

                sum_pro=sum(log_pro)

                normalized_posterior = [i/sum_pro for i in log_pro]    
                select_k = None
                if iter == (total_iter - 1):
                    select_k = normalized_posterior.index(max(normalized_posterior))

                else:
                    select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                if select_k == len(Topics):
                    choose_k = np.max(Topics) + 1
                else:
                    choose_k = Topics[select_k]

                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)

            count_k = []
            for k in Topics:
                if k in m_z.keys() and len(m_z[k]) == 0:
                    m_z.pop(k, None)
                    n_z.pop(k, None)
                    n_w.pop(k, None)
                    count_k.append(k)
            for k in count_k:
                Topics.remove(k)   

        from sklearn.metrics.cluster import normalized_mutual_info_score
        nmi_sample = []
        nmi_result = []
        for key, value in news_labels.items():
                nmi_sample.append(value)
                nmi_result.append(docID_assign_z[key])
        print("alpha ", alpha, "Topics ",len(Topics), " iter ",iter," ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
        alpha_NMI[alpha] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
        alpha_topics[alpha] = len(Topics)

iter:  0  ,docID  0
len(Topics)  0
iter:  0  ,docID  1000
len(Topics)  117
iter:  0  ,docID  2000
len(Topics)  133
iter:  0  ,docID  3000
len(Topics)  146
iter:  0  ,docID  4000
len(Topics)  154
iter:  0  ,docID  5000
len(Topics)  157
iter:  0  ,docID  6000
len(Topics)  160
iter:  0  ,docID  7000
len(Topics)  163
iter:  0  ,docID  8000
len(Topics)  168
iter:  0  ,docID  9000
len(Topics)  170
iter:  0  ,docID  10000
len(Topics)  173
iter:  0  ,docID  11000
len(Topics)  176
alpha  0.5 Topics  176  iter  0   0.8351219817002778
iter:  1  ,docID  0
len(Topics)  176
iter:  1  ,docID  1000
len(Topics)  176


KeyboardInterrupt: 

In [ ]:
print(alpha_NMI)
print(alpha_topics)

In [13]:
print(alpha_NMI)
print(alpha_topics)

{0.5: 0.8786823141048951, 1.0: 0.8816152438010685, 1.5: 0.8701236276428096, 2.0: 0.8799045538841577, 2.5: 0.883858852975355, 3.0: 0.8820719620645908, 3.5: 0.8817376081287808, 4.0: 0.8810678439981579, 4.5: 0.8863087695793594, 5.0: 0.8836477186424533}
{0.5: 175, 1.0: 207, 1.5: 214, 2.0: 222, 2.5: 233, 3.0: 236, 3.5: 245, 4.0: 250, 4.5: 253, 5.0: 259}


In [20]:
alpha_NMI = {}
alpha_topics = {}
for alpha in np.arange(0.1,1.1,0.1):
    alpha = np.around(alpha,decimals=1)
    docID_assign_z = {}
    m_z = {}
    n_z = {}
    n_w = {}
    n_b = {}
    Topics = []
    V = set()
    D = set()
    #alpha = 1
    beta = 0.02
    total_iter = 10
    for iter in range(total_iter):
        for docID in range(len(corpus)):
            if docID % 1000 == 0:
                print("iter: ",iter," ,docID ",docID)
                print("len(Topics) ",len(Topics))
            words = corpus[docID]
            D.discard(docID)
            if docID in docID_assign_z.keys():
                before_k = docID_assign_z[docID]
                m_z[before_k].discard(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        n_z[before_k][word] -= 1
                        n_w[before_k] -=1
            else:
                before_k = -1
            if len(D) == 0 and len(V) == 0:
                choose_k = 0
                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)
            else:
                log_pro = []
                for k in Topics:
                    pro_k = len(m_z[k])
                    if pro_k != 0:
                        i = 0
                        for biterm in words:
                            each_word = biterm.strip().split(',')
                            for word in each_word:
                                if word not in n_z[k].keys():
                                    n_z[k][word] = 0
                            for j in range(doc_biterm[docID][biterm]):
                                pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                i += 1

                    if pro_k == 0:
                        pro_k = sys.float_info.min
                    log_pro.append(pro_k)

                pro_new_k = alpha*(len(D))
                i = 0
                for biterm in words:
                    for j in range(doc_biterm[docID][biterm]):
                        pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                        i += 1

                if pro_new_k == 0:
                    pro_new_k = sys.float_info.min        
                log_pro.append(pro_new_k)

                sum_pro=sum(log_pro)

                normalized_posterior = [i/sum_pro for i in log_pro]    
                select_k = None
                if iter == (total_iter - 1):
                    select_k = normalized_posterior.index(max(normalized_posterior))

                else:
                    select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                if select_k == len(Topics):
                    choose_k = np.max(Topics) + 1
                else:
                    choose_k = Topics[select_k]

                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)

            count_k = []
            for k in Topics:
                if k in m_z.keys() and len(m_z[k]) == 0:
                    m_z.pop(k, None)
                    n_z.pop(k, None)
                    n_w.pop(k, None)
                    count_k.append(k)
            for k in count_k:
                Topics.remove(k)   

        from sklearn.metrics.cluster import normalized_mutual_info_score
        nmi_sample = []
        nmi_result = []
        for key, value in news_labels.items():
                nmi_sample.append(value)
                nmi_result.append(docID_assign_z[key])
        print("alpha ", alpha, "Topics ",len(Topics), " iter ",iter," ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
        alpha_NMI[alpha] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
        alpha_topics[alpha] = len(Topics)

iter:  0  ,docID  0
len(Topics)  0
iter:  0  ,docID  1000
len(Topics)  105
iter:  0  ,docID  2000
len(Topics)  116
iter:  0  ,docID  3000
len(Topics)  120
iter:  0  ,docID  4000
len(Topics)  126
iter:  0  ,docID  5000
len(Topics)  130
iter:  0  ,docID  6000
len(Topics)  133
iter:  0  ,docID  7000
len(Topics)  137
iter:  0  ,docID  8000
len(Topics)  140
iter:  0  ,docID  9000
len(Topics)  144
iter:  0  ,docID  10000
len(Topics)  146
iter:  0  ,docID  11000
len(Topics)  147
alpha  0.1 Topics  147  iter  0   0.8192126576469292
iter:  1  ,docID  0
len(Topics)  147
iter:  1  ,docID  1000
len(Topics)  146
iter:  1  ,docID  2000
len(Topics)  143
iter:  1  ,docID  3000
len(Topics)  144
iter:  1  ,docID  4000
len(Topics)  145
iter:  1  ,docID  5000
len(Topics)  144
iter:  1  ,docID  6000
len(Topics)  143
iter:  1  ,docID  7000
len(Topics)  142
iter:  1  ,docID  8000
len(Topics)  142
iter:  1  ,docID  9000
len(Topics)  140
iter:  1  ,docID  10000
len(Topics)  139
iter:  1  ,docID  11000
len(Topi

iter:  5  ,docID  6000
len(Topics)  157
iter:  5  ,docID  7000
len(Topics)  156
iter:  5  ,docID  8000
len(Topics)  159
iter:  5  ,docID  9000
len(Topics)  159
iter:  5  ,docID  10000
len(Topics)  159
iter:  5  ,docID  11000
len(Topics)  159
alpha  0.2 Topics  159  iter  5   0.8613719432437981
iter:  6  ,docID  0
len(Topics)  159
iter:  6  ,docID  1000
len(Topics)  157
iter:  6  ,docID  2000
len(Topics)  157
iter:  6  ,docID  3000
len(Topics)  154
iter:  6  ,docID  4000
len(Topics)  152
iter:  6  ,docID  5000
len(Topics)  152
iter:  6  ,docID  6000
len(Topics)  153
iter:  6  ,docID  7000
len(Topics)  155
iter:  6  ,docID  8000
len(Topics)  151
iter:  6  ,docID  9000
len(Topics)  151
iter:  6  ,docID  10000
len(Topics)  152
iter:  6  ,docID  11000
len(Topics)  152
alpha  0.2 Topics  152  iter  6   0.8601615169764301
iter:  7  ,docID  0
len(Topics)  152
iter:  7  ,docID  1000
len(Topics)  152
iter:  7  ,docID  2000
len(Topics)  153
iter:  7  ,docID  3000
len(Topics)  154
iter:  7  ,docID

alpha  0.4 Topics  174  iter  0   0.8234992869970489
iter:  1  ,docID  0
len(Topics)  174
iter:  1  ,docID  1000
len(Topics)  176
iter:  1  ,docID  2000
len(Topics)  178
iter:  1  ,docID  3000
len(Topics)  181
iter:  1  ,docID  4000
len(Topics)  178
iter:  1  ,docID  5000
len(Topics)  176
iter:  1  ,docID  6000
len(Topics)  176
iter:  1  ,docID  7000
len(Topics)  171
iter:  1  ,docID  8000
len(Topics)  168
iter:  1  ,docID  9000
len(Topics)  169
iter:  1  ,docID  10000
len(Topics)  170
iter:  1  ,docID  11000
len(Topics)  170
alpha  0.4 Topics  170  iter  1   0.8553796145647905
iter:  2  ,docID  0
len(Topics)  170
iter:  2  ,docID  1000
len(Topics)  168
iter:  2  ,docID  2000
len(Topics)  165
iter:  2  ,docID  3000
len(Topics)  161
iter:  2  ,docID  4000
len(Topics)  160
iter:  2  ,docID  5000
len(Topics)  160
iter:  2  ,docID  6000
len(Topics)  160
iter:  2  ,docID  7000
len(Topics)  161
iter:  2  ,docID  8000
len(Topics)  162
iter:  2  ,docID  9000
len(Topics)  162
iter:  2  ,docID  

iter:  6  ,docID  5000
len(Topics)  172
iter:  6  ,docID  6000
len(Topics)  170
iter:  6  ,docID  7000
len(Topics)  168
iter:  6  ,docID  8000
len(Topics)  170
iter:  6  ,docID  9000
len(Topics)  169
iter:  6  ,docID  10000
len(Topics)  172
iter:  6  ,docID  11000
len(Topics)  169
alpha  0.5 Topics  169  iter  6   0.8669217290586477
iter:  7  ,docID  0
len(Topics)  169
iter:  7  ,docID  1000
len(Topics)  172
iter:  7  ,docID  2000
len(Topics)  171
iter:  7  ,docID  3000
len(Topics)  167
iter:  7  ,docID  4000
len(Topics)  168
iter:  7  ,docID  5000
len(Topics)  170
iter:  7  ,docID  6000
len(Topics)  171
iter:  7  ,docID  7000
len(Topics)  172
iter:  7  ,docID  8000
len(Topics)  171
iter:  7  ,docID  9000
len(Topics)  170
iter:  7  ,docID  10000
len(Topics)  167
iter:  7  ,docID  11000
len(Topics)  170
alpha  0.5 Topics  170  iter  7   0.8683255786215845
iter:  8  ,docID  0
len(Topics)  170
iter:  8  ,docID  1000
len(Topics)  168
iter:  8  ,docID  2000
len(Topics)  168
iter:  8  ,docID

iter:  1  ,docID  11000
len(Topics)  190
alpha  0.7 Topics  189  iter  1   0.8636162791526129
iter:  2  ,docID  0
len(Topics)  189
iter:  2  ,docID  1000
len(Topics)  192
iter:  2  ,docID  2000
len(Topics)  193
iter:  2  ,docID  3000
len(Topics)  187
iter:  2  ,docID  4000
len(Topics)  189
iter:  2  ,docID  5000
len(Topics)  188
iter:  2  ,docID  6000
len(Topics)  188
iter:  2  ,docID  7000
len(Topics)  189
iter:  2  ,docID  8000
len(Topics)  190
iter:  2  ,docID  9000
len(Topics)  192
iter:  2  ,docID  10000
len(Topics)  192
iter:  2  ,docID  11000
len(Topics)  190
alpha  0.7 Topics  190  iter  2   0.8700841773942556
iter:  3  ,docID  0
len(Topics)  190
iter:  3  ,docID  1000
len(Topics)  190
iter:  3  ,docID  2000
len(Topics)  187
iter:  3  ,docID  3000
len(Topics)  190
iter:  3  ,docID  4000
len(Topics)  189
iter:  3  ,docID  5000
len(Topics)  187
iter:  3  ,docID  6000
len(Topics)  188
iter:  3  ,docID  7000
len(Topics)  187
iter:  3  ,docID  8000
len(Topics)  189
iter:  3  ,docID 

iter:  7  ,docID  4000
len(Topics)  205
iter:  7  ,docID  5000
len(Topics)  204
iter:  7  ,docID  6000
len(Topics)  206
iter:  7  ,docID  7000
len(Topics)  206
iter:  7  ,docID  8000
len(Topics)  204
iter:  7  ,docID  9000
len(Topics)  202
iter:  7  ,docID  10000
len(Topics)  203
iter:  7  ,docID  11000
len(Topics)  202
alpha  0.8 Topics  203  iter  7   0.8676119902776182
iter:  8  ,docID  0
len(Topics)  203
iter:  8  ,docID  1000
len(Topics)  202
iter:  8  ,docID  2000
len(Topics)  199
iter:  8  ,docID  3000
len(Topics)  195
iter:  8  ,docID  4000
len(Topics)  198
iter:  8  ,docID  5000
len(Topics)  196
iter:  8  ,docID  6000
len(Topics)  194
iter:  8  ,docID  7000
len(Topics)  194
iter:  8  ,docID  8000
len(Topics)  198
iter:  8  ,docID  9000
len(Topics)  195
iter:  8  ,docID  10000
len(Topics)  194
iter:  8  ,docID  11000
len(Topics)  195
alpha  0.8 Topics  194  iter  8   0.8673933394688027
iter:  9  ,docID  0
len(Topics)  194
iter:  9  ,docID  1000
len(Topics)  194
iter:  9  ,docID

iter:  2  ,docID  10000
len(Topics)  206
iter:  2  ,docID  11000
len(Topics)  208
alpha  1.0 Topics  207  iter  2   0.869040955517679
iter:  3  ,docID  0
len(Topics)  207
iter:  3  ,docID  1000
len(Topics)  204
iter:  3  ,docID  2000
len(Topics)  203
iter:  3  ,docID  3000
len(Topics)  205
iter:  3  ,docID  4000
len(Topics)  207
iter:  3  ,docID  5000
len(Topics)  210
iter:  3  ,docID  6000
len(Topics)  213
iter:  3  ,docID  7000
len(Topics)  208
iter:  3  ,docID  8000
len(Topics)  204
iter:  3  ,docID  9000
len(Topics)  205
iter:  3  ,docID  10000
len(Topics)  205
iter:  3  ,docID  11000
len(Topics)  202
alpha  1.0 Topics  202  iter  3   0.8696058175388325
iter:  4  ,docID  0
len(Topics)  202
iter:  4  ,docID  1000
len(Topics)  205
iter:  4  ,docID  2000
len(Topics)  205
iter:  4  ,docID  3000
len(Topics)  201
iter:  4  ,docID  4000
len(Topics)  197
iter:  4  ,docID  5000
len(Topics)  195
iter:  4  ,docID  6000
len(Topics)  193
iter:  4  ,docID  7000
len(Topics)  197
iter:  4  ,docID 

In [22]:
print(alpha_NMI.values())
print(alpha_NMI.keys())

dict_values([0.875357019833723, 0.8680878700250737, 0.8767182875014117, 0.8773262348982254, 0.8758216779885393, 0.8748901758378342, 0.8791601806130357, 0.8727822902554727, 0.8813728553555596, 0.8782347325334638])
dict_keys([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])


In [23]:
print(alpha_topics.values())
print(alpha_topics.keys())

dict_values([136, 142, 151, 159, 168, 169, 181, 206, 213, 214])
dict_keys([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])


In [17]:
print(alpha_NMI.values())
print(alpha_NMI.keys())

dict_values([0.8709004907479806, 0.8713827312891201, 0.8717560496741923, 0.8729172708546922, 0.8800098678237721, 0.8851168898932172, 0.880337035664144, 0.8745213072455894, 0.8731187657555207, 0.8741759900927224])
dict_keys([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])


In [18]:
print(alpha_topics.values())
print(alpha_topics.keys())

dict_values([120, 133, 142, 173, 173, 182, 170, 191, 195, 191])
dict_keys([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])


In [4]:
import timeit
total_batch = 16
NMI_batch = {}
start = 0
end = 0
topic_batch = {}
time_iteration_list = []
for batch_i in range(1,17):
    if batch_i == 16:
        end = int(len(corpus))
    else:
        end = int(len(corpus) / total_batch) * batch_i
    
    docID_assign_z = {}
    m_z = {}
    n_z = {}
    n_w = {}
    n_b = {}
    Topics = []
    V = set()
    D = set()
    alpha = 0.6
    beta = 0.02
    NMI_batch[batch_i] = 0
    topic_batch[batch_i] = 0
    
    time_start = timeit.default_timer()
    total_iter = 10
    for iter in range(total_iter):
        for docID in range(start,end):
            words = corpus[docID]
            D.discard(docID)
            if docID in docID_assign_z.keys():
                before_k = docID_assign_z[docID]
                m_z[before_k].discard(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        n_z[before_k][word] -= 1
                        n_w[before_k] -=1
            else:
                before_k = -1
            if len(D) == 0 and len(V) == 0:
                choose_k = 0
                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)
            else:
                log_pro = []
                for k in Topics:
                    pro_k = len(m_z[k])
                    if pro_k != 0:
                        i = 0
                        for biterm in words:
                            each_word = biterm.strip().split(',')
                            for word in each_word:
                                if word not in n_z[k].keys():
                                    n_z[k][word] = 0
                            for j in range(doc_biterm[docID][biterm]):
                                pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                i += 1

                    if pro_k == 0:
                        pro_k = sys.float_info.min
                    log_pro.append(pro_k)

                pro_new_k = alpha*(len(D))
                i = 0
                for biterm in words:
                    for j in range(doc_biterm[docID][biterm]):
                        pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                        i += 1

                if pro_new_k == 0:
                    pro_new_k = sys.float_info.min        
                log_pro.append(pro_new_k)

                sum_pro=sum(log_pro)

                normalized_posterior = [i/sum_pro for i in log_pro]    
                select_k = None
                if iter == (total_iter - 1):
                    select_k = normalized_posterior.index(max(normalized_posterior))

                else:
                    select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                if select_k == len(Topics):
                    choose_k = np.max(Topics) + 1
                else:
                    choose_k = Topics[select_k]

                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)

            count_k = []
            for k in Topics:
                if k in m_z.keys() and len(m_z[k]) == 0:
                    m_z.pop(k, None)
                    n_z.pop(k, None)
                    n_w.pop(k, None)
                    count_k.append(k)
            for k in count_k:
                Topics.remove(k)   
        if iter == 9:
            from sklearn.metrics.cluster import normalized_mutual_info_score
            nmi_sample = []
            nmi_result = []
            for key, value in news_labels.items():
                if key < end and key >= start:
                    nmi_sample.append(value)
                    nmi_result.append(docID_assign_z[key])
            NMI_batch[batch_i] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
            topic_batch[batch_i] = len(Topics)
            print("start ",start, " end ",end, "topics: ",len(Topics),"truth: ",len(np.unique(nmi_sample)), " NMI: ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
#     if batch_i != 1:
#         start = int(len(corpus) / total_batch) * (batch_i -1) + 1
    time_stop = timeit.default_timer()
    time_iteration_list.append(time_stop-time_start)
    print(time_iteration_list)

start  0  end  694 topics:  114 truth:  132  NMI:  0.8841135780125937
[27.687417765357054]
start  0  end  1388 topics:  129 truth:  143  NMI:  0.8744286405491407
[27.687417765357054, 63.82025185922781]
start  0  end  2082 topics:  144 truth:  148  NMI:  0.8603778629186858
[27.687417765357054, 63.82025185922781, 144.3630850847428]
start  0  end  2776 topics:  155 truth:  151  NMI:  0.8796346081185471
[27.687417765357054, 63.82025185922781, 144.3630850847428, 204.63872854688023]
start  0  end  3470 topics:  158 truth:  151  NMI:  0.8636027250256413
[27.687417765357054, 63.82025185922781, 144.3630850847428, 204.63872854688023, 260.6130805025887]
start  0  end  4164 topics:  161 truth:  152  NMI:  0.8730488100795639
[27.687417765357054, 63.82025185922781, 144.3630850847428, 204.63872854688023, 260.6130805025887, 343.7576088547047]
start  0  end  4858 topics:  176 truth:  152  NMI:  0.8709841883567744
[27.687417765357054, 63.82025185922781, 144.3630850847428, 204.63872854688023, 260.6130805

In [5]:
import timeit
total_batch = 16
NMI_batch = {}
start = 0
end = 0
topic_batch = {}
time_iteration_list = []
for batch_i in range(1,17):
    if batch_i == 16:
        end = int(len(corpus))
    else:
        end = int(len(corpus) / total_batch) * batch_i
    
    docID_assign_z = {}
    m_z = {}
    n_z = {}
    n_w = {}
    n_b = {}
    Topics = []
    V = set()
    D = set()
    alpha = 0.6
    beta = 0.02
    NMI_batch[batch_i] = 0
    topic_batch[batch_i] = 0
    
    time_start = timeit.default_timer()
    total_iter = 10
    for iter in range(total_iter):
        for docID in range(start,end):
            words = corpus[docID]
            D.discard(docID)
            if docID in docID_assign_z.keys():
                before_k = docID_assign_z[docID]
                m_z[before_k].discard(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        n_z[before_k][word] -= 1
                        n_w[before_k] -=1
            else:
                before_k = -1
            if len(D) == 0 and len(V) == 0:
                choose_k = 0
                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)
            else:
                log_pro = []
                for k in Topics:
                    pro_k = len(m_z[k])
                    if pro_k != 0:
                        i = 0
                        for biterm in words:
                            each_word = biterm.strip().split(',')
                            for word in each_word:
                                if word not in n_z[k].keys():
                                    n_z[k][word] = 0
                            for j in range(doc_biterm[docID][biterm]):
                                pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                i += 1

                    if pro_k == 0:
                        pro_k = sys.float_info.min
                    log_pro.append(pro_k)

                pro_new_k = alpha*(len(D))
                i = 0
                for biterm in words:
                    for j in range(doc_biterm[docID][biterm]):
                        pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                        i += 1

                if pro_new_k == 0:
                    pro_new_k = sys.float_info.min        
                log_pro.append(pro_new_k)

                sum_pro=sum(log_pro)

                normalized_posterior = [i/sum_pro for i in log_pro]    
                select_k = None
                if iter == (total_iter - 1):
                    select_k = normalized_posterior.index(max(normalized_posterior))

                else:
                    select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                if select_k == len(Topics):
                    choose_k = np.max(Topics) + 1
                else:
                    choose_k = Topics[select_k]

                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)

            count_k = []
            for k in Topics:
                if k in m_z.keys() and len(m_z[k]) == 0:
                    m_z.pop(k, None)
                    n_z.pop(k, None)
                    n_w.pop(k, None)
                    count_k.append(k)
            for k in count_k:
                Topics.remove(k)   
        if iter == 9:
            from sklearn.metrics.cluster import normalized_mutual_info_score
            nmi_sample = []
            nmi_result = []
            for key, value in news_labels.items():
                if key < end and key >= start:
                    nmi_sample.append(value)
                    nmi_result.append(docID_assign_z[key])
            NMI_batch[batch_i] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
            topic_batch[batch_i] = len(Topics)
            print("start ",start, " end ",end, "topics: ",len(Topics),"truth: ",len(np.unique(nmi_sample)), " NMI: ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
#     if batch_i != 1:
#         start = int(len(corpus) / total_batch) * (batch_i -1) + 1
    time_stop = timeit.default_timer()
    time_iteration_list.append(time_stop-time_start)
    print(time_iteration_list)

start  0  end  694 topics:  116 truth:  132  NMI:  0.8771256364967567
[26.498108096173382]
start  0  end  1388 topics:  132 truth:  143  NMI:  0.8679382030118227
[26.498108096173382, 59.47742560460756]
start  0  end  2082 topics:  141 truth:  148  NMI:  0.8671081750738531
[26.498108096173382, 59.47742560460756, 91.84906113916077]
start  0  end  2776 topics:  150 truth:  151  NMI:  0.872484574086654
[26.498108096173382, 59.47742560460756, 91.84906113916077, 125.93393405123788]
start  0  end  3470 topics:  158 truth:  151  NMI:  0.868296761210852
[26.498108096173382, 59.47742560460756, 91.84906113916077, 125.93393405123788, 164.09086516778916]
start  0  end  4164 topics:  167 truth:  152  NMI:  0.8682524971158333
[26.498108096173382, 59.47742560460756, 91.84906113916077, 125.93393405123788, 164.09086516778916, 211.71732525993139]
start  0  end  4858 topics:  171 truth:  152  NMI:  0.8724822673395431
[26.498108096173382, 59.47742560460756, 91.84906113916077, 125.93393405123788, 164.090865